# ChatGPT for Box Scores

Purpose: to answer questions and provide summative information about recent games or series from around baseball.

This program takes inputs of box scores and answers questions regarding noteworthy events from the games described. The box scores could be from around the MLB, minor leagues, or college baseball. It is not meant to provide in-depth analysis, but it can be an effective tool in keeping scouts, managers, and executives informed about the volume of games that they are unable to watch. The program takes an optional input of a pre-defined watchlist, which allows the user to request information about a particular selection of players. It can also be systematized to provide weekly updates of notable performances that might otherwise go unnoticed.

In [13]:
import openai as ai
import pandas as pd

# Set Openai key (key omitted for publication)
ai.api_key = api_key

'''
This example uses box scores of three MLB games from 9/14/23. In practice, the input would be a much...
larger set of games, beyond what could be efficiently followed without AI assistance.
These box scores were created using Statcast data in a separate R file.
'''
# Read in pitchers file 
BOS_NYYp = pd.read_csv('box_p_BOS_NYY.csv').drop(columns=['Unnamed: 0', 'player_id'])
MIN_CWSp = pd.read_csv('box_p_MIN_CWS.csv').drop(columns=['Unnamed: 0', 'player_id'])
TEX_TORp = pd.read_csv('box_p_TEX_TOR.csv').drop(columns=['Unnamed: 0', 'player_id'])

# Read in hitters file
BOS_NYYh = pd.read_csv('box_h_BOS_NYY.csv').drop(columns=['Unnamed: 0', 'player_id'])
MIN_CWSh = pd.read_csv('box_h_MIN_CWS.csv').drop(columns=['Unnamed: 0', 'player_id'])
TEX_TORh = pd.read_csv('box_h_TEX_TOR.csv').drop(columns=['Unnamed: 0', 'player_id'])

In [14]:
BOS_NYYh.head()

,Name,POS,Team,PA,H,BB,K,HR,Barrel,RC
0,Wilyer Abreu,LF,BOS,4,3,0,1,0,0,2.25
1,Justin Turner,DH,BOS,4,1,0,0,0,0,0.25
2,Gleyber Torres,2B,NYY,4,2,0,1,0,0,2.00
3,Rafael Devers,3B,BOS,4,1,1,1,0,0,1.67
4,Aaron Judge,RF,NYY,4,0,0,1,0,0,0.00


In [15]:
BOS_NYYp.head()

,Name,Team,IP,BF,K,BB,CSW_FB,CSW_BB,H,FB_avg
0,Chris Martin,BOS,1.0,5,0,0,0.23,NaN,2,95.9
1,Michael King,NYY,4.2,20,8,1,0.33,0.26,6,94.6
2,Greg Weissert,NYY,2.1,9,3,0,0.29,0.33,2,94.7
3,Tanner Houck,BOS,6.0,25,7,3,0.27,0.34,4,94.5
4,Garrett Whitlock,BOS,2.0,6,1,0,0.31,0.33,0,95.4


In [16]:
# Convert to json
BOS_NYYp = BOS_NYYp.to_json(orient='records')
MIN_CWSp = MIN_CWSp.to_json(orient='records')
TEX_TORp = TEX_TORp.to_json(orient='records')
BOS_NYYh = BOS_NYYh.to_json(orient='records')
MIN_CWSh = MIN_CWSh.to_json(orient='records')
TEX_TORh = TEX_TORh.to_json(orient='records')

#This list of box score files is an example of the input that the program could take.
box_scores = [BOS_NYYp, MIN_CWSp, TEX_TORp, BOS_NYYh, MIN_CWSh, TEX_TORh]

In [17]:
def chat_gpt_box(jsonList, prompt, watchlist = None):
    # Craft GPT model with messages
    messages = [
        {"role": "system", "content": '''You are given a box score from a baseball game. 
                                         Your job is to answer questions about how players performed in the games.'''},
        {"role": "system", "content": '''The columns for a pitcher box score are as follows. IP: innings pitched. 
                                         BF: batters faced. K: strikeouts. BB: walks. 
                                         CSW_FB: called strike + swing and miss rates on fastballs. 
                                         CSW_BB: called strike + swing and miss rates on breaking balls. 
                                         H: hits. FB_avg: the pitcher's average fastball velocity in the game.'''},
        {"role": "system", "content": '''When evaluating pitchers’ performances, look for high strikeouts, low hits, 
                                         and low walks. CSW_FB and CSW_BB are indicators of how the pitcher’s fastball 
                                         and breaking ball performed, respectively; 0.3 is around average.'''},
        {"role": "system", "content": '''The columns for a hitter box score are as follows: PA: plate appearances. 
                                         H: hits. BB: walks. K: strikeouts. HR: home runs. Barrel: Very well-hit balls. 
                                         RC: runs created. RC is the most important indicator of success. 
                                         Recording a barrel or HR is noteworthy.'''},
        {"role": "system", "content": '''Whenever you mention a player, follow it with his team in parenthesis. 
                                         Do not list out the stats unless they are specifically referenced.'''},
        {"role": "system", "content": '''You may be given a watchlist of names. If you are given a list of names, 
                                         then the user may ask for information about players on the watchlist. 
                                         If so, find the box score stats for the players on the watchlist. Some may 
                                         be pitchers, and some may be hitters. Use pitcher stats for the pitchers and 
                                         hitter stats for the hitters. Rather than just providing stats, provide a 
                                         brief written summary and make note of any notable stats.'''},
        {"role": "system", "content": str(watchlist) if watchlist else "No watchlist provided"}
    ]
    # Add each box score in the list
    for i in range(len(jsonList)):
        messages.append({"role": "assistant", "content": jsonList[i]})

    messages.append({"role": "user", "content": prompt})

    response = ai.ChatCompletion.create(
        model="gpt-3.5-turbo-16k",
        messages=messages,
        max_tokens=200
    )

    answer = response["choices"][0]["message"]["content"]
    tokens_used = response["usage"]["total_tokens"]

    result_dict = {"answer": answer, "tokens_used": tokens_used}
    return result_dict


In [18]:
def Chad2(jsonList, watchlist = None):

    user_input = input("Enter your prompt: ")
    result = chat_gpt_box(jsonList = jsonList, prompt = user_input, watchlist = watchlist)
    answer = result["answer"]
    print("Tokens used:", result["tokens_used"])
    cost = result["tokens_used"] / 1000 * .003
    print(f'cost: ${round(cost,4)}, or {round(cost * 100,1)} cents.')
    print(f'Prompt: {user_input}')
    
    return print(answer)

Prompt: What pitchers performed well yesterday?

In [51]:
Chad2(box_scores)

Tokens used: 4683
cost: $0.014, or 1.4 cents.
Prompt: What pitchers performed well yesterday?
Yesterday, there were several pitchers who performed well. Here are a few notable performances:

- Michael King (NYY) had a strong outing, pitching 4.2 innings with 8 strikeouts and only allowing 1 walk and 6 hits.
- Tanner Houck (BOS) also had a solid performance, pitching 6 innings with 7 strikeouts and only allowing 3 walks and 4 hits.
- Garrett Whitlock (BOS) had a good outing, pitching 2 innings with 1 strikeout and not allowing any hits or walks.
- Jose Urena (CWS) pitched 6.1 innings with 8 strikeouts, no walks, and 7 hits.
- Kenta Maeda (MIN) pitched 7 innings with 8 strikeouts and only allowed 1 walk and 4 hits.

These pitchers demonstrated strong command and were successful in limiting hits and walks while generating a good number of strikeouts.


Prompt: How was Maeda's fastball yesterday?

In [19]:
Chad2(box_scores)

Tokens used: 4587
cost: $0.0138, or 1.4 cents.
Prompt: How was Maeda's fastball yesterday?
Kenta Maeda's fastball averaged 90.7 mph in yesterday's game. This is relatively low compared to the average fastball velocity of pitchers. It's worth noting that his CSW_FB (called strike + swing and miss rate on fastballs) was 0.18, which is also on the lower side.


### Watchlist

This feature of the program can create more relevant insights, particularly when monitoring college and minor leagues. When considering possible draft picks, acquisitions, or promotions, it can be very helpful to maintain an active following of as many players as possible. 

In [24]:
# Create watchlist
myWatchlist = ['Triston Casas', 'Anthony Volpe', 'Gavin Sheets', 'Jose Urena', 'Cody Bradford']

Prompt: Who from the watchlist played well?

In [45]:
Chad2(box_scores, watchlist = myWatchlist)

Tokens used: 4674
cost: $0.014, or 1.4 cents.
Prompt: Who from the watchlist played well?
From the watchlist, Triston Casas (BOS) played well with 1 hit, 1 walk, and a run created of 0.25. Anthony Volpe (NYY) also had a solid performance with 2 hits, 1 walk, and a run created of 1.0. Gavin Sheets (CWS) had 1 hit and a run created of 0.25. Jose Urena (CWS) had a good outing as a pitcher, pitching 6.1 innings with 8 strikeouts, 0 walks, and allowing 7 hits. Cody Bradford (TEX) also had a strong pitching performance, striking out 4 batters in 2.2 innings with 1 walk and 1 hit allowed.


Prompt: How did Volpe play?

In [53]:
Chad2(box_scores, myWatchlist)

Tokens used: 4586
cost: $0.0138, or 1.4 cents.
Prompt: How did Volpe play?
Anthony Volpe (NYY) had a tough game at the plate. He went 0-for-4 with no hits in his four plate appearances. He did draw one walk and struck out once. Although he didn't have any notable stats in this game, it's important to note that one game does not define a player's performance.


### More with ChatGPT

I look forward to exploring additional uses of ChatGPT for MLB front offices. Possibilities include the following.

- SQL querying assistance for non-analysts
- Optimizing scouts' travel schedules, considering geography and noted prospects
- Summarizing long documents
- translating Spanish, Japanese, Korean, or other foreign language materials
- Mass communications
- Social media / league buzz monitoring

